In [28]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas
import base64

In [40]:

def decode_grib_file(encoded_data, output_path):
    """
    Decodes a base64 encoded GRIB file and saves it to a specified location.

    Parameters:
    - encoded_data (str): The base64 encoded string of the GRIB file content.
    - output_path (str): The path where the decoded GRIB file should be saved.

    Returns:
    - str: Path to the saved file.
    """

    # Split the received data into lines
    lines = encoded_data.strip().split("\n")

    # Extract only the message content lines, skipping the order numbers
    encoded_data_without_nrs = ''.join([lines[i] for i in range(len(lines)) if i % 3 == 1])

    
    # Decode the base64 encoded data to obtain the binary representation of the GRIB file
    decoded_data = base64.b64decode(encoded_data_without_nrs)
    

    # Save the decoded data to the specified path
    with open(output_path, 'wb') as file:
        file.write(decoded_data)

    return output_path

In [41]:
# Copy all messages received from the inreach
TEXT_RECIEVED = """
0
R1JJQgAAjAEAABwCYpr/gCFpAAoXCBcAAAEAJAoAAAAVAAABAAAgAP8AAAgABwBdwIEZQIAAjKCA4pAH0AfQQAAAAAAAAEQIAADCX1dsCDIATFBiX1VXPzth
0
1
UGFugnxfRVZob3aUmFxmbmhldZKXCwwNFiZIZ3kdISYlPENfXC4sLzUoHCJHADc3NzdHUklCAACMAQAAHAJimv+AIWkAChcIFwAAAQAwCgAAABUAAAEAACAA
1
2
/wAACAAHAF3AgRlAgACMoIDikAfQB9BAAAAAAAAARAgAAMJ2sGoIaDQAJGdzd2VYQ0BUan2PllpTaGN1h5CRMWl2fYV4hJEwKiYxRWF2fDxAOjElMUFZTD4z
2
3
LzAyMicANzc3N0dSSUIAAIwBAAAcAmKa/4AiaQAKFwgXAAABACQKAAAAFQAAAQAAIAD/AAAIAAcAXcCBGUCAAIyggOKQB9AH0EAAAAAAAABECAAAwnC2kghI
3
4
dKCRgIqHdVeCjHpxhoh/YmN4a2BcdYlgYm5fRU1hcDAuNUQ7Nj1HNzs5NikfLUBMQjhALhkAKQA3Nzc3R1JJQgAAjAEAABwCYpr/gCJpAAoXCBcAAAEAMAoA
4
5
AAAVAAABAAAgAP8AAAgABwBdwIEZQIAAjKCA4pAH0AfQQAAAAAAAAEQIAADCZS96CDYphIVtcXBzUVxqcHVhd0xbXldlX1dUaSleZ1VNWExTN0NQRjg1PzhO
5
6
TlNJRBkPOE1bW0AzIA4AADc3Nzc=
6

"""

# Define the path where the GRIB file should be saved
OUTPUT_PATH = './output.grb'

# Use the function to decode and save the GRIB file
df = decode_grib_file(TEXT_RECIEVED, OUTPUT_PATH)